In [1]:
from pathlib import Path
import gzip
import pickle
import requests
import torch

In [2]:
url = "https://github.com/pytorch/tutorials/raw/master/_static/"
filename = "mnist.pkl.gz"

In [3]:
data_path = Path("data") if Path("data").exists() else Path("../data")
path = data_path / "downloaded"/"vector_mnist"
path.mkdir(parents=True,exist_ok=True)

In [4]:
if not (path/filename).exists():
    content = requests.get(url+filename).content
    (path/filename).open("wb").write(content) 

In [5]:
!cd ../ && tree

.
├── data
│   └── downloaded
│       ├── vector-mnist
│       │   └── mnist.pkl.gz
│       └── vector_mnist
│           └── mnist.pkl.gz
├── notebooks
│   ├── lab01_pytorch.ipynb
│   └── lab01_pytorch_self.ipynb
└── text_recognizer
    ├── __init__.py
    ├── __pycache__
    │   ├── __init__.cpython-310.pyc
    │   └── __init__.cpython-38.pyc
    ├── data
    │   ├── __pycache__
    │   │   └── util.cpython-38.pyc
    │   └── util.py
    ├── metadata
    │   ├── __pycache__
    │   │   ├── mnist.cpython-310.pyc
    │   │   ├── mnist.cpython-38.pyc
    │   │   ├── shared.cpython-310.pyc
    │   │   └── shared.cpython-38.pyc
    │   ├── mnist.py
    │   └── shared.py
    ├── models
    │   ├── __init__.py
    │   ├── __pycache__
    │   │   ├── __init__.cpython-38.pyc
    │   │   └── mlp.cpython-38.pyc
    │   └── mlp.py
    └── util.py

13 directories, 20 files


In [6]:
datafile = (path/filename)

In [7]:
with gzip.open(datafile,"rb") as f:
    ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f,encoding="latin-1")


In [8]:
x_train, y_train, x_valid, y_valid = map(
    torch.tensor, (x_train, y_train, x_valid, y_valid)
)

In [9]:
x_train[0,::2].shape

torch.Size([392])

In [10]:
x_train.ndim

2

In [11]:
# re-execute this cell for more samples
import sys
sys.path.append("/Users/convergeml/engineering/fsdl-text-recognizer-2022-labs/lab01")

import random
import wandb  # just for some convenience methods that convert tensors to human-friendly datatypes
import text_recognizer.metadata.mnist as metadata # metadata module holds metadata separate from data

In [12]:
idx = random.randint(0, len(x_train))
example = x_train[idx]

print(y_train[idx])  # the label of the image
wandb.Image(example.reshape(*metadata.DIMS)).image  # the image itself

tensor(1)


In [13]:
import math

In [14]:
weights = torch.randn(784, 10) / math.sqrt(784)
weights.requires_grad_()
bias = torch.zeros(10, requires_grad=True)

In [15]:
def linear(x: torch.Tensor) -> torch.Tensor:
    return x @ weights + bias

In [16]:
def log_softmax(x: torch.Tensor) -> torch.Tensor:
    return x - torch.log(torch.sum(torch.exp(x), axis=1))[:, None]

def model(xb: torch.Tensor) -> torch.Tensor:
    return log_softmax(linear(xb))

In [17]:
#64*784 , 784 * 10
#64 * 10, 10
#64 * 10

In [18]:
bs = 64  # batch size

xb = x_train[0:bs]  # a batch of inputs
outs = model(xb)  # outputs on that batch

print(outs[0], outs.shape)  # outputs on the first element of the batch

tensor([-2.0293, -2.3486, -2.8051, -2.6019, -2.2160, -2.0180, -2.9038, -2.6996,
        -2.2800, -1.7594], grad_fn=<SelectBackward0>) torch.Size([64, 10])


In [19]:
def accuracy(out: torch.Tensor, yb: torch.Tensor) -> torch.Tensor:
    preds = torch.argmax(out, dim=1)
    return (preds == yb).float().mean()

In [20]:
yb = y_train[0:bs]

acc = accuracy(outs, yb)

print(acc)

tensor(0.1094)


In [21]:
try:
    acc.backward()
except RuntimeError as e:
    print(e)

element 0 of tensors does not require grad and does not have a grad_fn


In [22]:
def cross_entropy(output: torch.Tensor, target: torch.Tensor) -> torch.Tensor:
    return -output[range(target.shape[0]), target].mean()

loss_func = cross_entropy

In [23]:
print(type(outs),type(yb))

<class 'torch.Tensor'> <class 'torch.Tensor'>


In [24]:
print(
    loss_func(outs, yb), 
-torch.log(torch.tensor(1 / 10))
)

tensor(2.3546, grad_fn=<NegBackward0>) tensor(2.3026)


In [25]:
loss = loss_func(outs, yb)

In [26]:
loss.backward()

In [27]:
bias.grad

tensor([ 0.0180, -0.0228,  0.0265, -0.0286, -0.0283,  0.0598, -0.0227,  0.0160,
         0.0221, -0.0400])

In [28]:
lr = 0.5
epochs = 2
n, c = x_train.shape

for e in range(epochs):
     for ii in range((n - 1) // bs + 1):  # in batches of size bs, so roughly n / bs of them
        start_idx = ii * bs  # we are ii batches in, each of size bs
        end_idx = start_idx + bs  # and we want the next bs entires

        # pull batches from x and from y
        xb = x_train[start_idx:end_idx]
        yb = y_train[start_idx:end_idx]

        # run model
        pred = model(xb)

        # get loss
        loss = loss_func(pred, yb)
        print("happy:",loss)
        # calculate the gradients with a backwards pass
        loss.backward()

        with torch.no_grad():
            weights -= weights.grad*lr
            bias -= bias.grad*lr

            weights.grad.zero_()
            bias.grad.zero_()

happy: tensor(2.3546, grad_fn=<NegBackward0>)
happy: tensor(1.6536, grad_fn=<NegBackward0>)
happy: tensor(1.8955, grad_fn=<NegBackward0>)
happy: tensor(1.3255, grad_fn=<NegBackward0>)
happy: tensor(1.2411, grad_fn=<NegBackward0>)
happy: tensor(0.9473, grad_fn=<NegBackward0>)
happy: tensor(0.8276, grad_fn=<NegBackward0>)
happy: tensor(1.0709, grad_fn=<NegBackward0>)
happy: tensor(0.9315, grad_fn=<NegBackward0>)
happy: tensor(1.1125, grad_fn=<NegBackward0>)
happy: tensor(0.9486, grad_fn=<NegBackward0>)
happy: tensor(0.8854, grad_fn=<NegBackward0>)
happy: tensor(0.7753, grad_fn=<NegBackward0>)
happy: tensor(0.9009, grad_fn=<NegBackward0>)
happy: tensor(1.0211, grad_fn=<NegBackward0>)
happy: tensor(0.7140, grad_fn=<NegBackward0>)
happy: tensor(1.0250, grad_fn=<NegBackward0>)
happy: tensor(0.9668, grad_fn=<NegBackward0>)
happy: tensor(0.5213, grad_fn=<NegBackward0>)
happy: tensor(0.6565, grad_fn=<NegBackward0>)
happy: tensor(0.6628, grad_fn=<NegBackward0>)
happy: tensor(1.0339, grad_fn=<Neg

In [29]:
print(loss_func(model(xb), yb), accuracy(model(xb), yb))


tensor(0.0829, grad_fn=<NegBackward0>) tensor(1.)


In [30]:
# re-execute this cell for more samples
idx = random.randint(0, len(x_train))
example = x_train[idx:idx+1]

out = model(example)

print(out.argmax())
wandb.Image(example.reshape(28, 28)).image

tensor(0)


In [31]:
import torch.nn.functional as F
loss_func = F.cross_entropy

def model(xb):
    return xb @ weights + bias

In [32]:
print(loss_func(model(xb), yb), accuracy(model(xb), yb))  # should be unchanged from above!

tensor(0.0829, grad_fn=<NllLossBackward0>) tensor(1.)


In [33]:
from torch import nn

class MNISTLogistic(nn.Module):
    def __init__(self):
        super().__init__()
        self.weights = nn.Parameter(torch.randn(784,10)/math.sqrt(784))
        self.bias = nn.Parameter(torch.zeros(10))
    
    # def forward(self, xb: torch.Tensor):
    #     outs = xb @ self.weights + self.bias
    #     return outs


In [34]:
def forward(self, xb: torch.Tensor) -> torch.Tensor:
    return xb @ self.weights + self.bias

MNISTLogistic.forward = forward

In [35]:
model = MNISTLogistic()  # instantiated as an object
print(model(xb)[:4])  # callable like a function
loss = loss_func(model(xb), yb)  # composable like a function
loss.backward()  # we can still take gradients through it
print(model.weights.grad[::17,::2])  # and they show up in the .grad attribute

tensor([[-1.1783e-02, -3.0539e-01, -4.4323e-01, -3.7763e-01,  2.1540e-01,
         -7.2736e-02,  1.0071e-01,  6.8461e-01, -9.8071e-01, -2.0296e-01],
        [-2.0387e-01,  5.6281e-02, -5.4158e-01,  4.6071e-01,  1.3599e-01,
         -2.0744e-01, -9.2254e-01,  5.8820e-01, -1.2015e+00, -8.1617e-02],
        [ 1.7568e-01, -8.8359e-02, -4.8334e-01, -3.1407e-04,  4.5750e-01,
         -6.6327e-01, -5.4357e-01,  7.4714e-01,  2.0133e-02, -2.9589e-01],
        [ 2.9915e-01, -6.7942e-02, -5.6946e-01,  2.1351e-01,  5.6403e-01,
         -6.9438e-02, -5.1871e-01,  5.1175e-01, -7.6848e-01,  6.7242e-02]],
       grad_fn=<SliceBackward0>)
tensor([[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+

In [36]:
print(*list(model.parameters()),"\n")

Parameter containing:
tensor([[ 0.0363, -0.0251,  0.0149,  ...,  0.0222, -0.0090,  0.0636],
        [ 0.0069,  0.0207, -0.0274,  ..., -0.0096,  0.0467,  0.0107],
        [-0.0504,  0.0296,  0.0005,  ...,  0.0076,  0.0059, -0.0065],
        ...,
        [-0.0152,  0.0118,  0.0279,  ...,  0.0026,  0.0050,  0.0342],
        [-0.0167, -0.0463,  0.0168,  ..., -0.0831, -0.0235, -0.0313],
        [ 0.0767, -0.0261,  0.0512,  ...,  0.0231,  0.0057, -0.0075]],
       requires_grad=True) Parameter containing:
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], requires_grad=True) 



In [37]:
def fit():
    for e in epochs:
        for ii in range((n-1)//bs+1):
            start_idx = ii * bs
            end_idx = start_idx+bs
            xb = x_train[start_idx:end_idx]
            yb = y_train[start_idx:end_idx]
            pred = model(xb)
            loss = loss_func(pred,yb)
            
            loss.backward()
            with torch.no_grad():
                for p in model.parameters():
                    p-= lr * p.grad
                model.zero_grad()


In [38]:
print(accuracy(model(xb), yb))

tensor(0.0625)


In [39]:
import textwrap

print("torch.nn.Modules:", *textwrap.wrap(", ".join(torch.nn.modules.__all__)), sep="\n\t")

torch.nn.Modules:
	Module, Identity, Linear, Conv1d, Conv2d, Conv3d, ConvTranspose1d,
	ConvTranspose2d, ConvTranspose3d, Threshold, ReLU, Hardtanh, ReLU6,
	Sigmoid, Tanh, Softmax, Softmax2d, LogSoftmax, ELU, SELU, CELU, GLU,
	GELU, Hardshrink, LeakyReLU, LogSigmoid, Softplus, Softshrink,
	MultiheadAttention, PReLU, Softsign, Softmin, Tanhshrink, RReLU,
	L1Loss, NLLLoss, KLDivLoss, MSELoss, BCELoss, BCEWithLogitsLoss,
	NLLLoss2d, PoissonNLLLoss, CosineEmbeddingLoss, CTCLoss,
	HingeEmbeddingLoss, MarginRankingLoss, MultiLabelMarginLoss,
	MultiLabelSoftMarginLoss, MultiMarginLoss, SmoothL1Loss,
	GaussianNLLLoss, HuberLoss, SoftMarginLoss, CrossEntropyLoss,
	Container, Sequential, ModuleList, ModuleDict, ParameterList,
	ParameterDict, AvgPool1d, AvgPool2d, AvgPool3d, MaxPool1d, MaxPool2d,
	MaxPool3d, MaxUnpool1d, MaxUnpool2d, MaxUnpool3d, FractionalMaxPool2d,
	FractionalMaxPool3d, LPPool1d, LPPool2d, LocalResponseNorm,
	BatchNorm1d, BatchNorm2d, BatchNorm3d, InstanceNorm1d, InstanceNorm2d,

In [40]:
class MNISTLogistic(nn.Module):
    def __init__(self):
        super().__init__()
        self.lin = nn.Linear(784,10)
    
    def forward(self,xb:torch.Tensor):
        return self.lin(xb)

In [41]:
model = MNISTLogistic()
print(loss_func(model(xb), yb))  # loss is still close to 2.3

tensor(2.3291, grad_fn=<NllLossBackward0>)


In [42]:
list(model.children())

[Linear(in_features=784, out_features=10, bias=True)]

In [43]:
list(model.parameters())

[Parameter containing:
 tensor([[ 0.0226, -0.0180, -0.0316,  ...,  0.0177,  0.0009, -0.0053],
         [-0.0242,  0.0321,  0.0076,  ..., -0.0009, -0.0154,  0.0353],
         [-0.0336, -0.0246, -0.0280,  ..., -0.0067,  0.0250, -0.0034],
         ...,
         [ 0.0276, -0.0107, -0.0072,  ..., -0.0260, -0.0260, -0.0320],
         [-0.0224,  0.0280,  0.0223,  ...,  0.0351,  0.0129, -0.0117],
         [ 0.0040, -0.0044,  0.0248,  ...,  0.0307,  0.0256,  0.0175]],
        requires_grad=True),
 Parameter containing:
 tensor([-0.0037,  0.0038, -0.0177,  0.0186, -0.0013, -0.0273,  0.0199, -0.0218,
         -0.0265,  0.0299], requires_grad=True)]

In [44]:
from torch import optim

def configure_optimiser(model: nn.Module) -> optim.Optimizer:
    return optim.Adam(model.parameters(),lr=3e-4)


In [45]:
model = MNISTLogistic()
opt = configure_optimiser(model)

In [46]:
print("before training:", loss_func(model(xb), yb), sep="\n\t")


before training:
	tensor(2.2273, grad_fn=<NllLossBackward0>)


In [47]:
epochs = 10
for e in range(epochs):
    for ii in range((n-1)//bs+1):
        start_idx = ii*bs
        end_idx = start_idx + bs
        xb = x_train[start_idx:end_idx]
        yb = y_train[start_idx:end_idx]
        pred = model(xb)
        loss = loss_func(pred,yb) 
        loss.backward()

        opt.step()
        opt.zero_grad()

print("after training:", loss_func(model(xb), yb), sep="\n\t")

after training:
	tensor(0.4769, grad_fn=<NllLossBackward0>)


In [48]:
from text_recognizer.data.util import BaseDataset
train_ds = BaseDataset(x_train, y_train)

In [49]:
model = MNISTLogistic()
opt = configure_optimiser(model)


In [50]:
for epoch in range(epochs):
    for ii in range((n - 1) // bs + 1):
        xb, yb = train_ds[ii * bs: ii * bs + bs]  # xb and yb in one line!
        pred = model(xb)
        loss = loss_func(pred, yb)

        loss.backward()
        opt.step()
        opt.zero_grad()

print(loss_func(model(xb), yb))

tensor(0.4676, grad_fn=<NllLossBackward0>)


In [51]:
from torch.utils.data import DataLoader
train_ds = BaseDataset(x_train, y_train)
train_dataloader = DataLoader(train_ds, batch_size=bs)

In [52]:
def fit(self: nn.Module, train_dataloader: DataLoader):
    opt = configure_optimiser(self)

    for e in range(epochs):
        for xb, yb in train_dataloader:
            pred = self(xb)
            loss = loss_func(pred, yb)
            loss.backward()
            opt.step()
            opt.zero_grad()

MNISTLogistic.fit = fit

In [53]:
model = MNISTLogistic()
model.fit(train_dataloader)
print(loss_func(model(xb), yb))

tensor(0.4835, grad_fn=<NllLossBackward0>)


#### Swapping in another model

In [54]:
from text_recognizer.models.mlp import MLP
MLP.fit = fit  # attach our fitting loop

In [55]:
MLP.forward??

Signature: MLP.forward(self, x)
Docstring:
Defines the computation performed at every call.

Should be overridden by all subclasses.

.. note::
    Although the recipe for forward pass needs to be defined within
    this function, one should call the :class:`Module` instance afterwards
    instead of this since the former takes care of running the
    registered hooks while the latter silently ignores them.
Source:   
    def forward(self, x):
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.dropout(x)
        x = self.fc3(x)
        return x
File:      ~/engineering/fsdl-text-recognizer-2022-labs/lab01/text_recognizer/models/mlp.py
Type:      function


In [56]:
MLP.__init__??

Signature:
MLP.__init__(
    self,
    data_config: Dict[str, Any],
    args: argparse.Namespace = None,
) -> None
Docstring: Initializes internal Module state, shared by both nn.Module and ScriptModule.
Source:   
    def __init__(
        self,
        data_config: Dict[str, Any],
        args: argparse.Namespace = None,
    ) -> None:

        super().__init__()
        self.args = vars(args) if args is not None else {}
        self.data_config = data_config

        input_dim = np.prod(self.data_config["input_dims"])
        num_classes = len(self.data_config["mapping"])

        fc1_dim = self.args.get("fc1", FC1_DIM)
        fc2_dim = self.args.get("fc2", FC2_DIM)
        dropout_p = self.args.get("fc_dropout", FC_DROPOUT)

        self.fc1 = nn.Linear(input_dim, fc1_dim)
        self.dropout = nn.Dropout(dropout_p)
        self.fc2 = nn.Linear(fc1_dim, fc2_dim)
        self.fc3 = nn.Linear(fc2_dim, num_classes)
File:      ~/engineering/fsdl-text-recognizer-2022-labs/lab01/text_r

In [57]:
digits_to_9 = list(range(10))
data_config = {"input_dims": (784,), "mapping": {digit: str(digit) for digit in digits_to_9}}
data_config

{'input_dims': (784,),
 'mapping': {0: '0',
  1: '1',
  2: '2',
  3: '3',
  4: '4',
  5: '5',
  6: '6',
  7: '7',
  8: '8',
  9: '9'}}

In [58]:
model = MLP(data_config)
model

MLP(
  (fc1): Linear(in_features=784, out_features=1024, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc2): Linear(in_features=1024, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=10, bias=True)
)

In [59]:
model.fc1.weight.shape

torch.Size([1024, 784])

In [60]:
%%time
print("before training",loss_func(model(xb),yb))


before training tensor(2.2953, grad_fn=<NllLossBackward0>)
CPU times: user 7.84 ms, sys: 6.07 ms, total: 13.9 ms
Wall time: 13.1 ms


In [61]:
train_ds = BaseDataset(x_train, y_train)
train_dataloader = DataLoader(train_ds, batch_size=bs)
fit(model, train_dataloader=train_dataloader)

In [62]:
print("after training",loss_func(model(xb),yb))

after training tensor(0.0382, grad_fn=<NllLossBackward0>)


In [63]:
if torch.cuda.is_available():
    !nvidia-smi
else:
    print("☹️")

☹️


In [64]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

model.to(device)

loss_func(model(xb.to(device)), yb.to(device))

tensor(0.0881, grad_fn=<NllLossBackward0>)

In [65]:
def push_to_device(tensor):
    return tensor.to(device)

train_ds = BaseDataset(x_train, y_train, transform=push_to_device, target_transform=push_to_device)
train_dataloader = DataLoader(train_ds, batch_size=bs)

In [66]:
%%time

model = MLP(data_config)
model.to(device)

model.fit(train_dataloader)

print(loss_func(model(push_to_device(xb)), push_to_device(yb)))

tensor(0.0201, grad_fn=<NllLossBackward0>)
CPU times: user 2min 53s, sys: 6.65 s, total: 2min 59s
Wall time: 1min 41s


In [ ]:
class MNISTDataModule():
    url = "https://github.com/pytorch/tutorials/raw/master/_static/"
    filename = "mnist.pkl.gz"

    def __init__(self, dir, bs=32):
        self.dir = dir
        self.bs = bs
        self.path = self.dir/self.filename

    def prepare_data(self):
        if not (self.path).exists():
            content = requests.get(self.url+self.filename).content
            self.path.open("wb").write(content)
    
    def setup(self):
        with gzip.open(self.path,"rb") as f:
            ((x_train,y_train),(x_valid,y_valid), _) = pickle.load(f,encoding="latin-1") 
        
        x_train,y_train,x_valid,y_valid = map(torch.tensor,x_train,y_train,x_valid,y_valid)
        self.train_ds = BaseDataset(x_train, y_train, transform=push_to_device, target_transform=push_to_device)
        self.valid_ds = BaseDataset(x_valid, y_valid, transform=push_to_device, target_transform=push_to_device)

    def train_dataloader(self):
        return DataLoader(self.train_ds, batch_size=self.bs,shuffle=True)
    
    def valid_dataloader(self):
        return DataLoader(self.valid_ds, batch_size=self.bs,shuffle=True)

